# Practica 2B (Ascensores)
##### Alberto García Doménech - Pablo Daurell Marina (Grupo 03)

In [2]:
# Restriccion: Una persona por ascensor
# El último ascensor siempre es rápido (se  mueve por plantas pares)
# estado_ini = ( (personas), (ascensores) )
estado_ini = (
    (2,4,2,8,1), (2,0,1)
)

# estado_goal = (personas)
estado_goal = (3,11,12,1,9)

# Acciones
# Llamar ascensor persona (mueve un ascensor vacio a una planta)
# Subir ascensor persona (sube una persona dentro de un ascensor una planta) (comprobar si ascensor y persona estan en misma planta)
# Bajar ascensor persona (baja una persona dentro de un ascensor una planta) (comprobar si ascensor y persona estan en misma planta)

In [3]:
from search import *

class Ascensores(Problem):
    
    def __init__(self, initial, goal):
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        personas = list(estado[0])
        ascensores = list(estado[1])
        accs = list()
        for i, persona in enumerate(personas):
            for j, ascensor in enumerate(ascensores):
                if j == len(ascensores)-1: # ASCENSOR RAPIDO
                    if persona != ascensor and persona%2 == 0:
                        accs.append(("LlamarRapido", i, j)) # Persona i (en planta par) llama a ascensor rapido
                    if persona == ascensor and int(persona) >0:
                        accs.append(("BajarRapido" , i, j)) # Bajar persona i en asccensor rapido (j)
                    if persona == ascensor and int(persona) < 12:
                        accs.append(("SubirRapido", i, j)) # Subir persona i en ascensor rapido (j)
                else: # ASCENSOR NORMAL
                    if persona != ascensor:
                        accs.append(("Llamar", i, j))            # Persona i llama a asccensor j
                    if persona == ascensor and int(persona) > 0:
                        accs.append(("Bajar", i, j))             # Bajar persona i en ascensor j
                    if persona == ascensor and int(persona) < 12:
                        accs.append(("Subir", i, j))             # Subir persona i en ascensor j
            
        return accs
    
    def result(self, estado, accion):
        personas = list(estado[0])
        ascensores = list(estado[1])
        p = accion[1]
        a = accion[2]
        if accion[0] == "Llamar" or accion[0] == "LlamarRapido":
            ascensores[a] = int(personas[p])
        elif accion[0] == "Bajar":
            ascensores[a] -= 1
            personas[p] -= 1
        elif accion[0] == "Subir":
            ascensores[a] += 1
            personas[p] += 1
        elif accion[0] == "BajarRapido":
            ascensores[a] -= 2
            personas[p] -= 2
        elif accion[0] == "SubirRapido":
            ascensores[a] += 2
            personas[p] += 2
        return (tuple(personas), tuple(ascensores))
    
    def goal_test(self, estado):
        return estado[0] == self.goal
    
    def coste_de_aplicar_accion(self, estado, accion):
        personas = list(estado[0])
        ascensores = list(estado[1])
        p = accion[1]
        a = accion[2]
        if accion[0] == "Llamar":
            coste = abs(personas[p] - ascensores[a])
        elif accion[0] == "Bajar":
            coste = 1
        elif accion[0] == "Subir":
            coste = 1
        elif accion[0] == "LlamarRapido":
            coste = abs(personas[p] - ascensores[a])/2
        elif accion[0] == "BajarRapido":
            coste = 1/2
        elif accion[0] == "SubirRapido":
            coste = 1/2
        return coste


In [4]:
p = Ascensores(estado_ini, estado_goal)
p.initial

((2, 4, 2, 8, 1), (2, 0, 1))

In [5]:
p.goal

(3, 11, 12, 1, 9)

In [6]:
p.actions(p.initial)

[('Bajar', 0, 0),
 ('Subir', 0, 0),
 ('Llamar', 0, 1),
 ('LlamarRapido', 0, 2),
 ('Llamar', 1, 0),
 ('Llamar', 1, 1),
 ('LlamarRapido', 1, 2),
 ('Bajar', 2, 0),
 ('Subir', 2, 0),
 ('Llamar', 2, 1),
 ('LlamarRapido', 2, 2),
 ('Llamar', 3, 0),
 ('Llamar', 3, 1),
 ('LlamarRapido', 3, 2),
 ('Llamar', 4, 0),
 ('Llamar', 4, 1),
 ('BajarRapido', 4, 2),
 ('SubirRapido', 4, 2)]

In [7]:
p.result(p.initial, ("Llamar", 0, 0))

((2, 4, 2, 8, 1), (2, 0, 1))

In [8]:
p.coste_de_aplicar_accion(p.initial, ("LlamarRapido", 1, 0))

1.0

### Heurísticas

In [9]:
# Numero de personas en una planta incorrecta a la de destino 
def descolocados(node):
    res = 0
    goal = estado_goal
    for i in range(len(node.state[0])):
        if goal[i] != node.state[0][i]:
            res += 1
    return res

# Distancia de cada persona a su planta destino
def distancias(node):
    state = node.state[0]
    dist = 0
    goal = estado_goal
    for i in range(len(state)):
        if goal[i] != state[i]:
            dist += abs(state[i] - goal[i])
    return dist

# Distancia de cada persona a su planta destino (con ascensor rapido)
def distancias2(node):
    state = node.state[0]
    dist = 0
    goal = estado_goal
    for i in range(len(state)):
        if goal[i] != state[i]:
            dist += abs(state[i] - goal[i])/2
    return dist

### Búsqueda heurística

In [10]:
from search import best_first_graph_search

##### Dos personas, plantas pares

In [11]:
estado_ini = ((0,0),(0,0))
estado_goal = (4,10)
p = Ascensores(estado_ini, estado_goal)

In [25]:
%%time
astar_search(p, distancias2).solution()

Wall time: 32.9 ms


[('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1)]

In [26]:
%%time
astar_search(p, descolocados).solution()

Wall time: 121 ms


[('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1)]

In [29]:
%%time
best_first_graph_search(p, distancias2).solution()

Wall time: 2.96 ms


[('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0)]

In [30]:
%%time
best_first_graph_search(p, descolocados).solution()

Wall time: 12 ms


[('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1)]

##### Dos personas, plantas impares

In [33]:
estado_ini = ((1,3),(0,0))
estado_goal = (7,5)
p = Ascensores(estado_ini, estado_goal)

In [38]:
%%time
astar_search(p, distancias2).solution()

Wall time: 106 ms


[('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Llamar', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0)]

In [39]:
%%time
astar_search(p, descolocados).solution()

Wall time: 164 ms


[('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Llamar', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0)]

In [40]:
%%time
best_first_graph_search(p, distancias2).solution()

Wall time: 997 µs


[('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0)]

In [41]:
%%time
best_first_graph_search(p, descolocados).solution()

Wall time: 7.96 ms


[('Llamar', 0, 0),
 ('Bajar', 0, 0),
 ('Llamar', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0)]

##### Dos personas, plantas pares e impares

In [44]:
estado_ini = ((2,4),(0,0))
estado_goal = (7,1)
p = Ascensores(estado_ini, estado_goal)

In [45]:
%%time
astar_search(p, distancias2).solution()

Wall time: 107 ms


[('LlamarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Llamar', 1, 0),
 ('Bajar', 1, 0)]

In [46]:
%%time
astar_search(p, descolocados).solution()

Wall time: 217 ms


[('LlamarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('Llamar', 0, 0),
 ('Bajar', 1, 0),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('Llamar', 0, 0),
 ('Subir', 0, 0)]

In [47]:
%%time
best_first_graph_search(p, distancias2).solution()

Wall time: 997 µs


[('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('BajarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('Subir', 1, 0),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('Llamar', 0, 0),
 ('Subir', 0, 0)]

In [48]:
%%time
best_first_graph_search(p, descolocados).solution()

Wall time: 5.01 ms


[('LlamarRapido', 0, 1),
 ('BajarRapido', 0, 1),
 ('LlamarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('Subir', 1, 0),
 ('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0)]

##### Tres personas

In [52]:
estado_ini = ((2,4,1),(0,0))
estado_goal = (10,1,8)
p = Ascensores(estado_ini, estado_goal)

In [53]:
%%time
astar_search(p, distancias2).solution()

Wall time: 3.63 s


[('Llamar', 2, 0),
 ('Subir', 2, 0),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 2, 1),
 ('BajarRapido', 1, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('LlamarRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('Bajar', 1, 0)]

In [55]:
%%time
astar_search(p, descolocados).solution()

Wall time: 2min 2s


[('LlamarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('Llamar', 2, 0),
 ('Subir', 2, 0),
 ('Bajar', 1, 0),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('LlamarRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1)]

In [56]:
%%time
best_first_graph_search(p, distancias2).solution()

Wall time: 2.99 ms


[('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('BajarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('Subir', 1, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Llamar', 1, 0),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1)]

In [58]:
%%time
best_first_graph_search(p, descolocados).solution()

Wall time: 117 ms


[('LlamarRapido', 0, 1),
 ('BajarRapido', 0, 1),
 ('LlamarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('Subir', 1, 0),
 ('Bajar', 2, 0),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('LlamarRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1),
 ('SubirRapido', 0, 1)]

##### Dos personas, tres ascensores

In [74]:
estado_ini = ((2,4),(0,1,6))
estado_goal = (10,1,8)
p = Ascensores(estado_ini, estado_goal)

In [75]:
%%time
astar_search(p, distancias2).solution()

KeyboardInterrupt: 

In [ ]:
%%time
astar_search(p, descolocados).solution()

##### Cinco personas, dos ascensores

In [65]:
estado_ini = ((2,4,1,8,1),(0,0))
estado_goal = (3,11,12,1,9)
p = Ascensores(estado_ini, estado_goal)

In [66]:
%%time
best_first_graph_search(p, distancias2).solution();

Wall time: 60.8 ms


[('LlamarRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('Subir', 3, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Llamar', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Subir', 2, 0),
 ('Llamar', 3, 0),
 ('LlamarRapido', 1, 1),
 ('SubirRapido', 1, 1),
 ('LlamarRapido', 0, 1),
 ('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Llamar', 1, 0),
 ('Subir', 1, 0)]

In [67]:
%%time
best_first_graph_search(p, descolocados).solution();

Wall time: 2.19 s


[('LlamarRapido', 0, 1),
 ('BajarRapido', 0, 1),
 ('LlamarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('BajarRapido', 1, 1),
 ('LlamarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('BajarRapido', 3, 1),
 ('Subir', 3, 0),
 ('Bajar', 2, 0),
 ('Llamar', 3, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Subir', 4, 0),
 ('Llamar', 0, 0),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('SubirRapido', 2, 1),
 ('LlamarRapido', 0, 1),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Subir', 1, 0),
 ('Llamar', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0),
 ('Subir', 0, 0)]

##### Cinco personas, cinco ascensores

In [68]:
estado_ini = ((2,4,1,8,1),(0,4,4,8,0))
estado_goal = (3,11,12,1,9)
p = Ascensores(estado_ini, estado_goal)

In [69]:
%%time
best_first_graph_search(p, distancias2).solution();

Wall time: 232 ms


[('Bajar', 3, 3),
 ('Bajar', 3, 3),
 ('Bajar', 3, 3),
 ('Bajar', 3, 3),
 ('Bajar', 3, 1),
 ('Bajar', 3, 1),
 ('Bajar', 3, 1),
 ('Subir', 4, 1),
 ('Subir', 4, 1),
 ('Subir', 4, 1),
 ('Subir', 4, 3),
 ('Subir', 4, 3),
 ('Subir', 4, 3),
 ('Subir', 4, 3),
 ('Subir', 4, 3),
 ('Subir', 1, 2),
 ('Subir', 1, 2),
 ('Subir', 1, 2),
 ('Subir', 1, 2),
 ('Subir', 1, 2),
 ('Subir', 1, 3),
 ('Subir', 1, 3),
 ('Llamar', 2, 1),
 ('Subir', 2, 1),
 ('Subir', 2, 1),
 ('Subir', 2, 1),
 ('Subir', 2, 1),
 ('Subir', 2, 1),
 ('Subir', 2, 1),
 ('Subir', 2, 1),
 ('Subir', 2, 1),
 ('Subir', 2, 2),
 ('Subir', 2, 2),
 ('Subir', 2, 3),
 ('Llamar', 3, 1),
 ('Llamar', 3, 2),
 ('Llamar', 0, 3),
 ('Subir', 0, 3)]